# Note to Reviewer: 
Several feature engineering steps were completed throughout my data preprocessing and EDA steps. 
I've completed troubleshooting for previous open questions:
- why do all zctas have at least one flood? I made the choice early on to only include zctas with floods, but went back and added all. ZCTAs with no floods could add value to model.
- I added a new dataset, National hydrography dataset (ie streamlines) and calculated total channel lenght within each zcta.
- I also added a count of flood by season (fall, winter, and spring) to use in the model 

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn import tree
from sklearn.preprocessing import FunctionTransformer

from scipy import stats

In [2]:
data = pd.read_csv('data/02_data.csv', index_col=0)

data.isna().sum()

med_houseinc_12mon              0
household_public_assistance1    0
state_fips                      0
nflood_total                    0
flood_count_fall                0
flood_count_winter              0
flood_count_spring              0
nflood_2000s                    0
nflood_2010s                    0
flood_dur_hours_median          0
flood_dur_hours_min             0
flood_dur_hours_max             0
zcta                            0
channel_length_km               0
delta_floods                    0
dtype: int64

In [3]:
# keep zcta but remove from modeling data
data.index = data['zcta']

# drop columns not used in model
data_medianinc = data.drop(['zcta', 'state_fips', 'household_public_assistance1'], axis=1)

data_medianinc


,med_houseinc_12mon,nflood_total,flood_count_fall,flood_count_winter,flood_count_spring,nflood_2000s,nflood_2010s,flood_dur_hours_median,flood_dur_hours_min,flood_dur_hours_max,channel_length_km,delta_floods
zcta,,,,,,,,,,,,
35004,76683.0,1,0,0,0,1.0,0.0,1.967213,1.967213,1.967213,64.301,-1.000000
35005,55017.0,3,1,0,0,2.0,1.0,3.442623,1.967213,3.934426,94.617,-0.333333
35006,63521.0,2,0,1,0,0.0,1.0,3.196721,1.967213,4.426230,306.497,1.000000
35007,81351.0,6,1,0,1,1.0,4.0,2.459016,0.983607,8.852459,108.522,0.600000
35010,46730.0,5,0,0,1,2.0,2.0,1.721311,0.983607,8.360656,649.526,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
99919,64135.0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000
99921,59219.0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000
99922,53125.0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000


### Scale Data
I'll scale data later for each individual model. Planning on starting with RFR which doesn't need scaling (if fact, prefer not to for optimal interpretability). 

### Categorical data have already been removed in step 2 EDA. 

### split test/train
Split data using 80/20 train/test split for now. Planning to complete cross-validation check after first model built to check variances. 

In [4]:
# split test/train data
X = data_medianinc.drop('med_houseinc_12mon', axis=1)
y = data_medianinc['med_houseinc_12mon']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

y_train = np.log(y_train)
y_train

zcta
67661    10.965453
21701    11.490363
46760    10.994790
30817    10.793106
92629    11.689966
           ...    
67554    11.042057
59808    11.194124
1094     11.375822
71855    10.650934
77859    10.816112
Name: med_houseinc_12mon, Length: 24280, dtype: float64

In [ ]:

gb = GradientBoostingClassifier(

In [86]:
# Can we improve using different parameters?
# CV Grid Search for hyperparameter tuning
param_grid = {
    'random_state': [777],
    'max_depth': [5, 10, 20, None],
    'n_estimators': [100, 200, 300], 
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse']
}

grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=3)
grid_search.fit(X_train_fewer, y_train)
grid_search.best_params_


{'criterion': 'absolute_error',
 'max_depth': 5,
 'n_estimators': 300,
 'random_state': 777}

In [139]:
### still poor results. What dist is nflood anyway? 
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
fitdata = X_scaled['nflood_total']

gamma_vals = stats.gamma.fit(fitdata)
beta_vals = stats.beta.fit(fitdata)
lognormal_vals = stats.lognorm.fit(fitdata)
pareto_vals = stats.pareto.fit(fitdata)
pearson3_vals = stats.pearson3.fit(fitdata)
powerlaw_vals = stats.powerlaw.fit(fitdata)


print('normal: ', stats.kstest(fitdata, stats.norm.cdf))
print('gamma: ', stats.kstest(fitdata, 'gamma', args=gamma_vals))
print('beta: ', stats.kstest(fitdata, 'beta', args=beta_vals))
print('lognormal: ', stats.kstest(fitdata, 'lognorm', args=lognormal_vals))
print('pareto: ', stats.kstest(fitdata, 'pareto', args=pareto_vals))
print('pearson3: ', stats.kstest(fitdata, 'pearson3', args=pearson3_vals))
print('powerlaw: ', stats.kstest(fitdata, 'powerlaw', args=powerlaw_vals))


pearson3_vals
# fitdata

/Users/kealiepretzlav/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/kealiepretzlav/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/kealiepretzlav/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:6111: RuntimeWarning: invalid value encountered in log
  lndata = np.log(data - loc)


normal:  KstestResult(statistic=0.30057165786331397, pvalue=0.0, statistic_location=-0.5227570730737843, statistic_sign=-1)
gamma:  KstestResult(statistic=0.42347049351542304, pvalue=0.0, statistic_location=-0.3600284864453145, statistic_sign=-1)
beta:  KstestResult(statistic=0.3628525415875003, pvalue=0.0, statistic_location=-0.5227570730737843, statistic_sign=1)
lognormal:  KstestResult(statistic=0.6247069497347992, pvalue=0.0, statistic_location=-0.3600284864453145, statistic_sign=-1)
pareto:  KstestResult(statistic=0.36285460116635365, pvalue=0.0, statistic_location=-0.5227570730737843, statistic_sign=1)
pearson3:  KstestResult(statistic=0.36285454151330143, pvalue=0.0, statistic_location=-0.5227570730737843, statistic_sign=1)
powerlaw:  KstestResult(statistic=0.37683672702296894, pvalue=0.0, statistic_location=-0.3600284864453145, statistic_sign=-1)


(2.977369555266118, -2.8261544112678908e-16, 0.7782204970849549)